## 1. 编程目标
编写本程序的目的旨在自动获取某家企业最近几年内的财务数据，并完成数据可视化。具体来说，可分为如下几步：

1. 通过Akshare模块获取某一特定个股的财务数据，包括资产负债表、利润表和现金流量表；

2. 编写一个模块使得特定年的财务数据表格能够直观的展示；


3. 提取其中较为有意义的数据进行可视化

   
4. 计算特殊的指标变化情况。

## 2. 获取某一个股的财报数据
基于Akshare模块，通过同花顺网站获取数据。

In [2]:
import akshare as ak
# 目标股票代码
stock_code = '603986'

同花顺数据库提供了**按照报告期、按照季度和按照年度**排列的财务数据，这里我们将分开获取不同时间跨度的财务信息。

通过indicator关键字访问不同的时间跨度的财务信息。indicator="按报告期"; choice of {"按报告期", "按年度", "按单季度"}

### 2.1 利润表
#### 按照年排序的利润表


In [5]:
benefit_ths_df = ak.stock_financial_benefit_ths(symbol=stock_code, indicator="按年度")
print(benefit_ths_df)

     报告期 报表核心指标      *净利润  *营业总收入  *营业总成本 *归属于母公司所有者的净利润 *扣除非经常性损益后的净利润  \
0   2023            1.61亿  57.61亿  57.89亿          1.61亿       2740.06万   
1   2022           20.53亿  81.30亿  60.31亿         20.53亿         19.10亿   
2   2021           23.37亿  85.10亿  61.19亿         23.37亿         22.24亿   
3   2020            8.80亿  44.97亿  39.53亿          8.81亿          5.55亿   
4   2019            6.05亿  32.03亿  25.98亿          6.07亿          5.66亿   
5   2018            4.04亿  22.46亿  18.58亿          4.05亿          3.61亿   
6   2017            3.98亿  20.30亿  16.54亿          3.97亿          3.32亿   
7   2016            1.75亿  14.89亿  13.28亿          1.76亿          1.51亿   
8   2015            1.56亿  11.89亿  10.26亿          1.58亿          1.42亿   
9   2014         9825.25万   9.47亿   8.61亿       9812.25万       7494.61万   
10  2013         6735.04万   7.89亿   7.25亿       6735.04万       5796.35万   
11  2012         6333.30万   8.35亿   7.58亿       6333.30万       6237.16万   
12  2011         1787.53万

现在让我们看一看利润表包含哪些栏目

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
benefit_ths_df.columns

Index(['报告期', '报表核心指标', '*净利润', '*营业总收入', '*营业总成本', '*归属于母公司所有者的净利润',
       '*扣除非经常性损益后的净利润', '报表全部指标', '一、营业总收入', '其中：营业收入', '二、营业总成本', '其中：营业成本',
       '营业税金及附加', '销售费用', '管理费用', '研发费用', '财务费用', '其中：利息费用', '利息收入', '资产减值损失',
       '信用减值损失', '加：公允价值变动收益', '投资收益', '其中：联营企业和合营企业的投资收益', '资产处置收益', '其他收益',
       '三、营业利润', '加：营业外收入', '减：营业外支出', '其中：非流动资产处置损失', '四、利润总额', '减：所得税费用',
       '五、净利润', '（一）持续经营净利润', '归属于母公司所有者的净利润', '少数股东损益', '扣除非经常性损益后的净利润',
       '六、每股收益', '（一）基本每股收益', '（二）稀释每股收益', '七、其他综合收益', '归属母公司所有者的其他综合收益',
       '八、综合收益总额', '归属于母公司股东的综合收益总额', '归属于少数股东的综合收益总额'],
      dtype='object')

先画个直方图看看，不过在这之前，需要将数据转化一下。

In [28]:
def money_str_to_float(money_str):
    if type(money_str)==str:
        if '万' in money_str:
            return float(money_str.replace('万', '')) * 1
        elif '亿' in money_str:
            return float(money_str.replace('亿', '')) * 10000
        else:
            return money_str
    else:
        return money_str

benefit_df_yearly = benefit_ths_df.map(money_str_to_float)
benefit_df_yearly['（二）稀释每股收益']

0     0.24
1     3.09
2     3.53
3     1.36
4     1.44
5     1.43
6     1.41
7     1.06
8     2.10
9     1.31
10    0.90
11    0.86
12    0.29
Name: （二）稀释每股收益, dtype: object

In [29]:
from pyecharts import options as opts
from pyecharts.charts import Bar

c = (
    Bar()
    .add_xaxis([str(x) for x in benefit_df_yearly['报告期'].to_list()
               ])
    .add_yaxis("营业总收入",  benefit_df_yearly['*营业总收入'].to_list())
    .add_yaxis("净利润", benefit_df_yearly['*净利润'].to_list())
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
        title_opts=opts.TitleOpts(title="Bar-旋转X轴标签", subtitle="解决标签名字过长的问题"),
    )
    .render("bar_rotate_xaxis_label.html")
)
